# Infer-19-Decision-Expert-Systems : Decisions Robustes et Systemes Experts

**Serie** : Programmation Probabiliste avec Infer.NET (19/20)  
**Duree estimee** : 50 minutes  
**Prerequis** : Notebooks 14-18 (Decision Theory)

---

## Objectifs

- Comprendre les **systemes experts** et leur architecture
- Appliquer le critere **Minimax** pour decisions robustes
- Implementer le critere **Minimax Regret**
- Gerer l'**incertitude sur les probabilites** elles-memes

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-18-Decision-Value-Information](Infer-18-Decision-Value-Information.ipynb) | [Infer-20-Decision-Sequential](Infer-20-Decision-Sequential.ipynb) |

---

## 1. Systemes Experts : Architecture et Historique

### Definition

Un **systeme expert** est un programme informatique qui simule le raisonnement d'un expert humain dans un domaine specifique.

### Historique

| Systeme | Annee | Domaine | Innovation |
|---------|-------|---------|------------|
| **DENDRAL** | 1965 | Chimie | Premier systeme expert |
| **MYCIN** | 1976 | Medical | Regles avec facteurs de certitude |
| **PROSPECTOR** | 1979 | Geologie | Reseaux bayesiens |
| **R1/XCON** | 1982 | Configuration | Premier systeme commercial |

### Architecture classique

```
            +-------------------+
            |   Base de        |
            |   Connaissances  |
            +--------+----------+
                     |
                     v
+----------+   +----+-----+   +-----------+
| Interface|<->| Moteur   |<->| Interface |
| Expert   |   |Inference |   |Utilisateur|
+----------+   +----+-----+   +-----------+
                     |
                     v
            +--------+----------+
            |   Base de Faits   |
            +-------------------+
```

In [1]:
// Installation Infer.NET
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;

Console.WriteLine("Infer.NET charge !");

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

Infer.NET charge !


In [2]:
// Mini-systeme expert de diagnostic

public class MiniExpertSystem
{
    // Base de connaissances : P(symptome|maladie)
    private Dictionary<(string maladie, string symptome), double> _likelihoods;
    private Dictionary<string, double> _priors;
    private Dictionary<string, double> _utilities; // U(traitement, maladie)
    
    public MiniExpertSystem()
    {
        // Priors sur les maladies
        _priors = new Dictionary<string, double>
        {
            { "Grippe", 0.30 },
            { "Rhume", 0.50 },
            { "Covid", 0.15 },
            { "Allergie", 0.05 }
        };
        
        // Likelihoods P(symptome|maladie)
        _likelihoods = new Dictionary<(string, string), double>
        {
            { ("Grippe", "fievre"), 0.9 },
            { ("Grippe", "toux"), 0.8 },
            { ("Grippe", "fatigue"), 0.9 },
            { ("Grippe", "eternuements"), 0.3 },
            
            { ("Rhume", "fievre"), 0.3 },
            { ("Rhume", "toux"), 0.6 },
            { ("Rhume", "fatigue"), 0.4 },
            { ("Rhume", "eternuements"), 0.9 },
            
            { ("Covid", "fievre"), 0.85 },
            { ("Covid", "toux"), 0.75 },
            { ("Covid", "fatigue"), 0.8 },
            { ("Covid", "eternuements"), 0.2 },
            
            { ("Allergie", "fievre"), 0.05 },
            { ("Allergie", "toux"), 0.3 },
            { ("Allergie", "fatigue"), 0.2 },
            { ("Allergie", "eternuements"), 0.95 },
        };
    }
    
    public Dictionary<string, double> Diagnose(List<string> symptomesObserves)
    {
        // Calcul bayesien des posterieurs
        var posteriors = new Dictionary<string, double>();
        double evidence = 0;
        
        foreach (var maladie in _priors.Keys)
        {
            double likelihood = 1.0;
            foreach (var symptome in new[] { "fievre", "toux", "fatigue", "eternuements" })
            {
                double p = _likelihoods[(maladie, symptome)];
                if (symptomesObserves.Contains(symptome))
                    likelihood *= p;
                else
                    likelihood *= (1 - p);
            }
            posteriors[maladie] = _priors[maladie] * likelihood;
            evidence += posteriors[maladie];
        }
        
        // Normaliser
        foreach (var m in _priors.Keys)
            posteriors[m] /= evidence;
        
        return posteriors;
    }
}

var expert = new MiniExpertSystem();
var symptomes = new List<string> { "fievre", "toux", "fatigue" };

Console.WriteLine($"Symptomes observes : {string.Join(", ", symptomes)}\n");
Console.WriteLine("Diagnostic (posterieurs) :");

var posteriors = expert.Diagnose(symptomes);
foreach (var kv in posteriors.OrderByDescending(x => x.Value))
{
    Console.WriteLine($"  {kv.Key}: {kv.Value:P1}");
}

Symptomes observes : fievre, toux, fatigue

Diagnostic (posterieurs) :
  Grippe: 67,7 %
  Covid: 30,5 %
  Rhume: 1,8 %
  Allergie: 0,0 %



(8,40): warning CS0169: Le champ 'MiniExpertSystem._utilities' n'est jamais utilisé



## 2. Decision sous Incertitude Severe

### Le probleme

Jusqu'ici, nous avons suppose connaitre les probabilites exactes.
Mais souvent, nous avons de l'**incertitude sur les probabilites elles-memes** !

### Exemples

- Nouvelle maladie : pas de donnees epidemiologiques
- Technologie emergente : pas d'historique
- Expert incertain : "entre 20% et 40%"

### Approches

| Approche | Description | Usage |
|----------|-------------|-------|
| **Bayesienne** | Prior sur les probabilites | Si prior disponible |
| **Credal sets** | Ensemble de distributions | Bornes connues |
| **Minimax** | Pire cas | Decision conservative |
| **Minimax regret** | Minimiser le regret maximal | Compromis |

## 3. Critere Minimax

### Definition

> Choisir l'action qui **maximise l'utilite dans le pire cas**.

$$a^* = \arg\max_a \min_s U(a, s)$$

### Proprietes

- **Conservateur** : Ne prend aucun risque
- **Pessimiste** : Suppose que la nature est adversaire
- **Adapte** : Quand les erreurs sont tres couteuses

In [3]:
// Critere Minimax

public class MinimaxDecision
{
    public static (string bestAction, double worstCaseUtility) Solve(
        string[] actions, 
        string[] states, 
        double[,] utilities)
    {
        string bestAction = null;
        double maxOfMins = double.NegativeInfinity;
        
        Console.WriteLine("Analyse Minimax :\n");
        Console.WriteLine("Action          | " + string.Join(" | ", states.Select(s => $"{s,8}")) + " | Min");
        Console.WriteLine(new string('-', 60));
        
        for (int a = 0; a < actions.Length; a++)
        {
            double minUtility = double.PositiveInfinity;
            var utilities_a = new List<double>();
            
            for (int s = 0; s < states.Length; s++)
            {
                utilities_a.Add(utilities[a, s]);
                minUtility = Math.Min(minUtility, utilities[a, s]);
            }
            
            Console.WriteLine($"{actions[a],-15} | " + 
                string.Join(" | ", utilities_a.Select(u => $"{u,8:N0}")) + 
                $" | {minUtility,6:N0}");
            
            if (minUtility > maxOfMins)
            {
                maxOfMins = minUtility;
                bestAction = actions[a];
            }
        }
        
        return (bestAction, maxOfMins);
    }
}

// Exemple : Decision d'investissement
var actions = new[] { "Obligations", "Actions", "Immobilier", "Cash" };
var states = new[] { "Recession", "Stable", "Croissance" };
var utilities = new double[,]
{
    { 30000, 40000, 50000 },    // Obligations : stable
    { -20000, 50000, 120000 },  // Actions : volatile
    { -10000, 30000, 80000 },   // Immobilier
    { 20000, 20000, 20000 }     // Cash : garanti
};

var (best, worstCase) = MinimaxDecision.Solve(actions, states, utilities);
Console.WriteLine();
Console.WriteLine($"=> Decision Minimax : {best} (pire cas = {worstCase:N0})");

Analyse Minimax :

Action          | Recession |   Stable | Croissance | Min
------------------------------------------------------------
Obligations     |   30 000 |   40 000 |   50 000 | 30 000
Actions         |  -20 000 |   50 000 |  120 000 | -20 000
Immobilier      |  -10 000 |   30 000 |   80 000 | -10 000
Cash            |   20 000 |   20 000 |   20 000 | 20 000

=> Decision Minimax : Obligations (pire cas = 30 000)


## 4. Critere Minimax Regret

### Motivation

Minimax est souvent trop conservateur. Une alternative est de minimiser le **regret** maximal.

### Definition du regret

$$\text{Regret}(a, s) = \max_{a'} U(a', s) - U(a, s)$$

Le regret mesure "combien j'aurais pu faire mieux si j'avais su".

### Critere Minimax Regret

$$a^* = \arg\min_a \max_s \text{Regret}(a, s)$$

Minimiser le regret dans le pire cas.

In [4]:
// Critere Minimax Regret

public class MinimaxRegretDecision
{
    public static (string bestAction, double maxRegret, double[,] regretMatrix) Solve(
        string[] actions, 
        string[] states, 
        double[,] utilities)
    {
        int nA = actions.Length;
        int nS = states.Length;
        
        // 1. Calculer le meilleur outcome pour chaque etat
        double[] bestPerState = new double[nS];
        for (int s = 0; s < nS; s++)
        {
            bestPerState[s] = double.NegativeInfinity;
            for (int a = 0; a < nA; a++)
                bestPerState[s] = Math.Max(bestPerState[s], utilities[a, s]);
        }
        
        // 2. Calculer la matrice de regret
        var regret = new double[nA, nS];
        for (int a = 0; a < nA; a++)
            for (int s = 0; s < nS; s++)
                regret[a, s] = bestPerState[s] - utilities[a, s];
        
        // 3. Trouver l'action avec regret max minimal
        string bestAction = null;
        double minOfMaxRegrets = double.PositiveInfinity;
        
        for (int a = 0; a < nA; a++)
        {
            double maxRegretForAction = 0;
            for (int s = 0; s < nS; s++)
                maxRegretForAction = Math.Max(maxRegretForAction, regret[a, s]);
            
            if (maxRegretForAction < minOfMaxRegrets)
            {
                minOfMaxRegrets = maxRegretForAction;
                bestAction = actions[a];
            }
        }
        
        return (bestAction, minOfMaxRegrets, regret);
    }
}

// Meme exemple d'investissement
var (bestRegret, maxReg, regretMatrix) = MinimaxRegretDecision.Solve(actions, states, utilities);

Console.WriteLine("\nMatrice de Regret :\n");
Console.WriteLine("Action          | " + string.Join(" | ", states.Select(s => $"{s,8}")) + " | MaxReg");
Console.WriteLine(new string('-', 65));

for (int a = 0; a < actions.Length; a++)
{
    double maxR = 0;
    var regrets = new List<double>();
    for (int s = 0; s < states.Length; s++)
    {
        regrets.Add(regretMatrix[a, s]);
        maxR = Math.Max(maxR, regretMatrix[a, s]);
    }
    Console.WriteLine($"{actions[a],-15} | " + 
        string.Join(" | ", regrets.Select(r => $"{r,8:N0}")) + 
        $" | {maxR,6:N0}");
}

Console.WriteLine();
Console.WriteLine($"=> Decision Minimax Regret : {bestRegret} (max regret = {maxReg:N0})");


Matrice de Regret :

Action          | Recession |   Stable | Croissance | MaxReg
-----------------------------------------------------------------
Obligations     |        0 |   10 000 |   70 000 | 70 000
Actions         |   50 000 |        0 |        0 | 50 000
Immobilier      |   40 000 |   20 000 |   40 000 | 40 000
Cash            |   10 000 |   30 000 |  100 000 | 100 000

=> Decision Minimax Regret : Immobilier (max regret = 40 000)


## 5. Comparaison des Criteres

### Resume

| Critere | Decision | Philosophie |
|---------|----------|-------------|
| **Max EU** | Utilite esperee max | Probabilites connues |
| **Minimax** | Pire cas max | Pessimiste |
| **Minimax Regret** | Regret max min | Compromis |
| **Maximin** | = Minimax (autre nom) | - |

In [5]:
// Comparaison des trois criteres

// Probabilites hypothetiques
var probs = new[] { 0.25, 0.50, 0.25 }; // Recession, Stable, Croissance

Console.WriteLine("=== Comparaison des Criteres ===\n");
Console.WriteLine($"Hypothese de probabilites : {string.Join(", ", states.Zip(probs, (s, p) => $"{s}={p:P0}"))}\n");

// 1. Max EU
string bestEU = null;
double maxEU = double.NegativeInfinity;

Console.WriteLine("1. Maximisation de l'Utilite Esperee :");
for (int a = 0; a < actions.Length; a++)
{
    double eu = 0;
    for (int s = 0; s < states.Length; s++)
        eu += probs[s] * utilities[a, s];
    
    Console.WriteLine($"   E[U({actions[a]})] = {eu:N0}");
    if (eu > maxEU)
    {
        maxEU = eu;
        bestEU = actions[a];
    }
}
Console.WriteLine($"   => Decision : {bestEU}\n");

// 2. Minimax
Console.WriteLine("2. Minimax (max du pire cas) :");
var (minmaxBest, minmaxVal) = MinimaxDecision.Solve(actions, states, utilities);
Console.WriteLine($"\n   => Decision : {minmaxBest}\n");

// 3. Minimax Regret
Console.WriteLine("3. Minimax Regret :");
var (regretBest, regretVal, _) = MinimaxRegretDecision.Solve(actions, states, utilities);
Console.WriteLine($"   => Decision : {regretBest}\n");

// Resume
Console.WriteLine("=== Resume ===\n");
Console.WriteLine($"Max EU         : {bestEU}");
Console.WriteLine($"Minimax        : {minmaxBest}");
Console.WriteLine($"Minimax Regret : {regretBest}");

=== Comparaison des Criteres ===

Hypothese de probabilites : Recession=25 %, Stable=50 %, Croissance=25 %

1. Maximisation de l'Utilite Esperee :
   E[U(Obligations)] = 40 000
   E[U(Actions)] = 50 000
   E[U(Immobilier)] = 32 500
   E[U(Cash)] = 20 000
   => Decision : Actions

2. Minimax (max du pire cas) :
Analyse Minimax :

Action          | Recession |   Stable | Croissance | Min
------------------------------------------------------------
Obligations     |   30 000 |   40 000 |   50 000 | 30 000
Actions         |  -20 000 |   50 000 |  120 000 | -20 000
Immobilier      |  -10 000 |   30 000 |   80 000 | -10 000
Cash            |   20 000 |   20 000 |   20 000 | 20 000

   => Decision : Obligations

3. Minimax Regret :
   => Decision : Immobilier

=== Resume ===

Max EU         : Actions
Minimax        : Obligations
Minimax Regret : Immobilier


## 6. Gamma-Maximin : Compromis entre Optimisme et Pessimisme

### Idee

Plutot que d'etre completement pessimiste (minimax) ou optimiste (maximax), on peut faire une moyenne ponderee.

### Formule Hurwicz

$$H(a) = \gamma \cdot \max_s U(a,s) + (1-\gamma) \cdot \min_s U(a,s)$$

Ou γ ∈ [0,1] est le coefficient d'optimisme :
- γ = 0 : Minimax (pessimiste)
- γ = 1 : Maximax (optimiste)
- γ = 0.5 : Neutre

In [6]:
// Critere Hurwicz (Gamma-Maximin)

public static (string best, double value) HurwiczCriterion(
    string[] actions, double[,] utilities, double gamma)
{
    int nA = actions.Length;
    int nS = utilities.GetLength(1);
    
    string best = null;
    double maxH = double.NegativeInfinity;
    
    for (int a = 0; a < nA; a++)
    {
        double minU = double.PositiveInfinity;
        double maxU = double.NegativeInfinity;
        
        for (int s = 0; s < nS; s++)
        {
            minU = Math.Min(minU, utilities[a, s]);
            maxU = Math.Max(maxU, utilities[a, s]);
        }
        
        double H = gamma * maxU + (1 - gamma) * minU;
        
        if (H > maxH)
        {
            maxH = H;
            best = actions[a];
        }
    }
    
    return (best, maxH);
}

Console.WriteLine("Critere Hurwicz selon gamma :\n");
Console.WriteLine("gamma  | Decision       | H(a*)");
Console.WriteLine("-------|----------------|--------");

for (double g = 0; g <= 1.0; g += 0.2)
{
    var (best, val) = HurwiczCriterion(actions, utilities, g);
    Console.WriteLine($"{g,5:F1}  | {best,-14} | {val,6:N0}");
}

Console.WriteLine();
Console.WriteLine("=> Differents niveaux d'optimisme menent a differentes decisions.");

Critere Hurwicz selon gamma :

gamma  | Decision       | H(a*)
-------|----------------|--------
  0,0  | Obligations    | 30 000
  0,2  | Obligations    | 34 000
  0,4  | Obligations    | 38 000
  0,6  | Actions        | 64 000
  0,8  | Actions        | 92 000
  1,0  | Actions        | 120 000

=> Differents niveaux d'optimisme menent a differentes decisions.


## 7. Robustesse aux Erreurs de Modelisation

### Probleme

Meme si on utilise l'approche bayesienne, le modele peut etre **mal specifie**.

### Techniques de robustesse

1. **Analyse de sensibilite** : Tester la decision sur une plage de parametres
2. **Ensembles de probabilites** : Considerer toutes les distributions dans un ensemble
3. **Decision robuste** : Optimiser le pire cas sur l'ensemble

In [7]:
// Analyse de sensibilite

Console.WriteLine("=== Analyse de Sensibilite ===\n");
Console.WriteLine("P(Recession) | Meilleure action | EU");
Console.WriteLine("-------------|------------------|--------");

for (double pRecession = 0.1; pRecession <= 0.5; pRecession += 0.1)
{
    // Redistribuer le reste entre Stable et Croissance
    double pStable = (1 - pRecession) * 0.65;  // 65% du reste
    double pCroissance = (1 - pRecession) * 0.35;
    var p = new[] { pRecession, pStable, pCroissance };
    
    string best = null;
    double maxEU = double.NegativeInfinity;
    
    for (int a = 0; a < actions.Length; a++)
    {
        double eu = 0;
        for (int s = 0; s < states.Length; s++)
            eu += p[s] * utilities[a, s];
        
        if (eu > maxEU)
        {
            maxEU = eu;
            best = actions[a];
        }
    }
    
    Console.WriteLine($"{pRecession,12:P0} | {best,-16} | {maxEU,6:N0}");
}

Console.WriteLine();
Console.WriteLine("=> La decision change quand P(Recession) devient assez elevee.");

=== Analyse de Sensibilite ===

P(Recession) | Meilleure action | EU
-------------|------------------|--------
        10 % | Actions          | 65 050
        20 % | Actions          | 55 600
        30 % | Actions          | 46 150
        40 % | Obligations      | 38 100
        50 % | Obligations      | 36 750

=> La decision change quand P(Recession) devient assez elevee.


## 8. Systeme Expert avec Decision Robuste

In [8]:
// Systeme expert medical avec decision robuste

public class RobustMedicalDecision
{
    // Maladies possibles
    public string[] Maladies { get; } = { "Benin", "Modere", "Grave" };
    
    // Traitements possibles
    public string[] Traitements { get; } = { "Aucun", "Leger", "Intensif" };
    
    // Utilites (incluant effets secondaires)
    public double[,] Utilites { get; } = new double[,]
    {
        // Benin, Modere, Grave
        { 100, 60, 10 },    // Aucun traitement
        { 90, 80, 40 },     // Traitement leger
        { 70, 85, 80 }      // Traitement intensif
    };
    
    public void Analyze(double[] posteriors)
    {
        Console.WriteLine("=== Analyse Decision Medicale ===\n");
        Console.WriteLine($"Posterieurs : {string.Join(", ", Maladies.Zip(posteriors, (m, p) => $"{m}={p:P0}"))}\n");
        
        // 1. Max EU
        Console.WriteLine("1. Maximisation EU :");
        string bestEU = null;
        double maxEU = double.NegativeInfinity;
        
        for (int t = 0; t < Traitements.Length; t++)
        {
            double eu = 0;
            for (int m = 0; m < Maladies.Length; m++)
                eu += posteriors[m] * Utilites[t, m];
            
            Console.WriteLine($"   E[U({Traitements[t]})] = {eu:F1}");
            if (eu > maxEU)
            {
                maxEU = eu;
                bestEU = Traitements[t];
            }
        }
        Console.WriteLine($"   => {bestEU}\n");
        
        // 2. Minimax (pire cas)
        Console.WriteLine("2. Minimax (conservateur) :");
        var (minmaxBest, _) = MinimaxDecision.Solve(Traitements, Maladies, Utilites);
        Console.WriteLine($"\n   => {minmaxBest}\n");
        
        // 3. Minimax Regret
        Console.WriteLine("3. Minimax Regret :");
        var (regretBest, _, _) = MinimaxRegretDecision.Solve(Traitements, Maladies, Utilites);
        Console.WriteLine($"   => {regretBest}\n");
        
        // Recommandation
        Console.WriteLine("=== Recommandation ===\n");
        
        // Si forte probabilite de cas grave, etre conservateur
        if (posteriors[2] > 0.3)
        {
            Console.WriteLine($"ATTENTION : P(Grave) = {posteriors[2]:P0} > 30%");
            Console.WriteLine($"Recommandation conservative : {minmaxBest}");
        }
        else
        {
            Console.WriteLine($"Recommandation basee sur EU : {bestEU}");
        }
    }
}

var robustDecision = new RobustMedicalDecision();

// Scenario 1 : Faible risque
Console.WriteLine("\n========== SCENARIO 1 : Faible risque ==========");
robustDecision.Analyze(new[] { 0.70, 0.25, 0.05 });

// Scenario 2 : Risque eleve
Console.WriteLine("\n========== SCENARIO 2 : Risque eleve ==========");
robustDecision.Analyze(new[] { 0.20, 0.40, 0.40 });


========== SCENARIO 1 : Faible risque ==========
=== Analyse Decision Medicale ===

Posterieurs : Benin=70 %, Modere=25 %, Grave=5 %

1. Maximisation EU :
   E[U(Aucun)] = 85,5
   E[U(Leger)] = 85,0
   E[U(Intensif)] = 74,2
   => Aucun

2. Minimax (conservateur) :
Analyse Minimax :

Action          |    Benin |   Modere |    Grave | Min
------------------------------------------------------------
Aucun           |      100 |       60 |       10 |     10
Leger           |       90 |       80 |       40 |     40
Intensif        |       70 |       85 |       80 |     70

   => Intensif

3. Minimax Regret :
   => Intensif

=== Recommandation ===

Recommandation basee sur EU : Aucun

========== SCENARIO 2 : Risque eleve ==========
=== Analyse Decision Medicale ===

Posterieurs : Benin=20 %, Modere=40 %, Grave=40 %

1. Maximisation EU :
   E[U(Aucun)] = 48,0
   E[U(Leger)] = 66,0
   E[U(Intensif)] = 80,0
   => Intensif

2. Minimax (conservateur) :
Analyse Minimax :

Action          |    Ben

## 9. Exercice : Systeme Expert de Diagnostic

### Enonce

Construisez un mini-systeme expert pour diagnostiquer des problemes informatiques.

Symptomes : lenteur, ecran_bleu, bruit_ventilateur, surchauffe
Causes possibles : virus, disque_plein, RAM_defaillante, surchauffe_CPU

1. Definissez les probabilites conditionnelles
2. Implementez le diagnostic bayesien
3. Proposez une action avec critere minimax regret

In [9]:
// Exercice : Systeme expert informatique (template)

var causes = new[] { "virus", "disque_plein", "RAM_defaillante", "surchauffe_CPU" };
var symptomes_possible = new[] { "lenteur", "ecran_bleu", "bruit_ventilateur", "surchauffe" };

// Priors (a completer avec des valeurs realistes)
var priors_cause = new Dictionary<string, double>
{
    { "virus", 0.30 },
    { "disque_plein", 0.35 },
    { "RAM_defaillante", 0.15 },
    { "surchauffe_CPU", 0.20 }
};

// Likelihoods P(symptome|cause) - a completer
var likelihoods_info = new Dictionary<(string, string), double>
{
    { ("virus", "lenteur"), 0.8 },
    { ("virus", "ecran_bleu"), 0.1 },
    { ("virus", "bruit_ventilateur"), 0.1 },
    { ("virus", "surchauffe"), 0.2 },
    
    { ("disque_plein", "lenteur"), 0.9 },
    { ("disque_plein", "ecran_bleu"), 0.05 },
    { ("disque_plein", "bruit_ventilateur"), 0.4 },
    { ("disque_plein", "surchauffe"), 0.1 },
    
    { ("RAM_defaillante", "lenteur"), 0.6 },
    { ("RAM_defaillante", "ecran_bleu"), 0.9 },
    { ("RAM_defaillante", "bruit_ventilateur"), 0.1 },
    { ("RAM_defaillante", "surchauffe"), 0.2 },
    
    { ("surchauffe_CPU", "lenteur"), 0.5 },
    { ("surchauffe_CPU", "ecran_bleu"), 0.3 },
    { ("surchauffe_CPU", "bruit_ventilateur"), 0.95 },
    { ("surchauffe_CPU", "surchauffe"), 0.98 },
};

// Symptomes observes
var symptomes_obs = new List<string> { "lenteur", "bruit_ventilateur" };

Console.WriteLine($"Symptomes observes : {string.Join(", ", symptomes_obs)}\n");

// Diagnostic bayesien
var posteriors_cause = new Dictionary<string, double>();
double evidence_total = 0;

foreach (var cause in causes)
{
    double likelihood = 1.0;
    foreach (var symptome in symptomes_possible)
    {
        double p = likelihoods_info[(cause, symptome)];
        likelihood *= symptomes_obs.Contains(symptome) ? p : (1 - p);
    }
    posteriors_cause[cause] = priors_cause[cause] * likelihood;
    evidence_total += posteriors_cause[cause];
}

// Normaliser
foreach (var cause in causes)
    posteriors_cause[cause] /= evidence_total;

Console.WriteLine("Diagnostic (posterieurs) :");
foreach (var kv in posteriors_cause.OrderByDescending(x => x.Value))
    Console.WriteLine($"  {kv.Key}: {kv.Value:P1}");

Console.WriteLine($"\n=> Cause la plus probable : {posteriors_cause.OrderByDescending(x => x.Value).First().Key}");

Symptomes observes : lenteur, bruit_ventilateur

Diagnostic (posterieurs) :
  disque_plein: 84,8 %
  virus: 13,6 %
  surchauffe_CPU: 1,0 %
  RAM_defaillante: 0,6 %

=> Cause la plus probable : disque_plein


## 10. Resume

| Concept | Application |
|---------|-------------|
| **Systemes experts** | Diagnostic, configuration, conseil |
| **Minimax** | Decisions critiques, erreurs couteuses |
| **Minimax Regret** | Compromis optimisme/pessimisme |
| **Hurwicz** | Parametre d'optimisme ajustable |
| **Robustesse** | Analyse de sensibilite, ensembles de probabilites |

---

## Pour aller plus loin

| Si vous voulez... | Consultez... |
|-------------------|-------------|
| Decisions sequentielles | [Infer-20-Decision-Sequential](Infer-20-Decision-Sequential.ipynb) |
| Reinforcement Learning | Serie RL dans `MyIA.AI.Notebooks/RL/` |

---

## Prochaine etape

Dans [Infer-20-Decision-Sequential](Infer-20-Decision-Sequential.ipynb), nous verrons :

- Les Processus de Decision Markoviens (MDPs)
- L'iteration de valeur et de politique
- Le reward shaping
- Les bandits multi-bras et l'indice de Gittins
- Introduction aux POMDPs

---

## References

- Shortliffe (1976) : MYCIN: Computer-Based Medical Consultations
- Wald (1950) : Statistical Decision Functions
- Savage (1951) : The Theory of Statistical Decision